# Iteration 3: Sample length of stay on the ASU

## Plain English Summary

In the model length of stay is determined not just by patient type, but also patient destination post ASU.  For example Early Supported Discharge greatly reduces length of stay.  Now that we have sampled the destination we can sample the length of stay from Lognormal distributions.



## Notebook imports

In [1]:
from IPython.display import Markdown, display
import math

## LLM Prompt

In [2]:
display(Markdown("inputs/03_iteration.md"))

### Main command

Modify the model to add in four new `acute_treatment` methods to the ASU class each patient type i.e. Stroke, TIA, Complex Neurological, and other.  Prefix the method names with the patient type e.g. `stroke_acute_treatment`

Show all of the model code: this includes all patient generator functions in the ASU class.

### General model logic

Add the following logic

1. After a patient has been generated call `acute_treatment` as a simpy process. Pass the patient type to the method.  Do not use the yield statement when calling this method.
2. sample a length of stay (measured in days) for treatment of patients
3. yield a simpy timeout equal to the length of stay for treatment

The model should print out useful information after each event.

### Simulation inputs

the length of stay (measured in days) for treatment of patients in the ASU follow source specific distributions

if patient type = Stroke and post_asu_destination = Rehab then Lognormal: mean 7.4 and standard deviation 8.6

if patient type = Stroke and post_asu_destination = ESD then Lognormal: mean 4.6 and standard deviation 4.8

if patient type = Stroke and post_asu_destination = Other then Lognormal: mean 7.0 and standard deviation 8.7

if patient type = TIA then Lognormal: mean 1.8 and standard deviation 5.0

if patient type = Complex Neurological then Lognormal: mean 4.0 and standard deviation 5.0

if patient type = Other then Lognormal: mean 3.8 and standard deviation 5.2

The mean and standard deviation of the Lognormal distributions must be converted to the mean and standard deviation of the underlying normal distribution.




## Date and time of input to LLM

> 1500 2nd March 2024

## Observations on the LLM Output

* Note the input above had to be modified in the following ways:
  * "Do not use the yield statement when calling this method." - originally the code included the `yield` keyword before calling the acute_treatment process code. This meant arrival generator code was incorrect (wrong IAT).

* The `acute_treatment` functions appear to have been created correctly.

* Sampling from the lognormal distribution is not correct. I.e. it does not correctly convert from Lognormal mean and std to the underlying normal distribution.

```python
length_of_stay = np.random.lognormal(mean=np.log(7.4), sigma=np.log(8.6))
```

* `stroke_acute_treatment` includes correct conditional logic for sub-stroke types.

* Redundant code in `stroke_patient_generator` i.e. it did not need to use conditional logic. However it does work.

```python
if post_asu_destination == "Rehab":
    self.env.process(self.stroke_acute_treatment("Rehab"))
elif post_asu_destination == "ESD":
    self.env.process(self.stroke_acute_treatment("ESD"))
else:
    self.env.process(self.stroke_acute_treatment("Other"))
```


## Sources reported by the LLM

* None given - Perplexity appears to have stopped providing them.

## LLM Output

In [3]:
import simpy
import numpy as np

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_acute_treatment(self, patient_type):
        if patient_type == "Rehab":
            length_of_stay = np.random.lognormal(mean=np.log(7.4), sigma=np.log(8.6))
        elif patient_type == "ESD":
            length_of_stay = np.random.lognormal(mean=np.log(4.6), sigma=np.log(4.8))
        else:
            length_of_stay = np.random.lognormal(mean=np.log(7.0), sigma=np.log(8.7))
        
        yield self.env.timeout(length_of_stay)
        print(f"Stroke patient {self.patient_count} finished treatment at {self.env.now} days")

    def tia_acute_treatment(self):
        length_of_stay = np.random.lognormal(mean=np.log(1.8), sigma=np.log(5.0))
        yield self.env.timeout(length_of_stay)
        print(f"TIA patient {self.patient_count} finished treatment at {self.env.now} days")

    def neuro_acute_treatment(self):
        length_of_stay = np.random.lognormal(mean=np.log(4.0), sigma=np.log(5.0))
        yield self.env.timeout(length_of_stay)
        print(f"Complex Neuro patient {self.patient_count} finished treatment at {self.env.now} days")

    def other_acute_treatment(self):
        length_of_stay = np.random.lognormal(mean=np.log(3.8), sigma=np.log(5.2))
        yield self.env.timeout(length_of_stay)
        print(f"Other patient {self.patient_count} finished treatment at {self.env.now} days")

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")
            if post_asu_destination == "Rehab":
                self.env.process(self.stroke_acute_treatment("Rehab"))
            elif post_asu_destination == "ESD":
                self.env.process(self.stroke_acute_treatment("ESD"))
            else:
                self.env.process(self.stroke_acute_treatment("Other"))

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.tia_acute_treatment())

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.neuro_acute_treatment())

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")
            self.env.process(self.other_acute_treatment())

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)

Complex Neuro patient 1 arrived at 0.8411807933691057 days
Complex Neuro patient 2 arrived at 1.038903599379167 days
TIA patient 3 arrived at 1.2670147129001517 days
Other patient 4 arrived at 1.47265847560886 days
Stroke patient 5 arrived at 1.5306227757474404 days and will go to Other
Stroke patient 6 arrived at 2.437473120480691 days and will go to Other
Stroke patient 7 arrived at 2.6891374640469765 days and will go to Other
Complex Neuro patient 7 finished treatment at 2.9361102882105663 days
Other patient 8 arrived at 3.0036344755314732 days
Complex Neuro patient 9 arrived at 3.5727457479986393 days
Stroke patient 10 arrived at 4.030650322923932 days and will go to Other
TIA patient 11 arrived at 4.216761039464449 days
TIA patient 11 finished treatment at 5.029795753534557 days
Other patient 12 arrived at 5.385802005671605 days
Stroke patient 13 arrived at 5.503425950323075 days and will go to Other
Other patient 14 arrived at 5.804512218614123 days
Stroke patient 14 finished tre

## Testing

* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods




### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [4]:
M = 10_000_000

In [5]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [6]:
test_1(M)

Stroke patient 1 arrived at 2.3235620647305493 days and will go to Other
Stroke patient 2 arrived at 3.4299112979359725 days and will go to Other
Stroke patient 2 finished treatment at 3.753640395911404 days
Stroke patient 2 finished treatment at 4.0207502652304035 days
Stroke patient 3 arrived at 9.045263022915162 days and will go to Rehab
Stroke patient 4 arrived at 10.753966947190978 days and will go to Other
Stroke patient 5 arrived at 10.954502132670678 days and will go to Other
Stroke patient 5 finished treatment at 11.819716216152704 days
Stroke patient 6 arrived at 12.389667026222202 days and will go to Other
Stroke patient 6 finished treatment at 12.597814134396685 days
Stroke patient 7 arrived at 13.150247913291983 days and will go to ESD
Stroke patient 8 arrived at 13.598289704773858 days and will go to Other
Stroke patient 8 finished treatment at 13.619376882768982 days
Stroke patient 9 arrived at 14.226217959162165 days and will go to Other
Stroke patient 10 arrived at 15.

### Extreme value test 2

* The code is again modified so that all patient typs have a mean inter-arrival rate of $M$

**Result: PASS**

In [7]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [8]:
test_2(M)

0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [9]:
TEN_DAYS = 10

In [10]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    


In [11]:
test_3(TEN_DAYS)

Other patient 1 arrived at 2.885731185145616 days
TIA patient 2 arrived at 3.3646938776854 days
Stroke patient 3 arrived at 3.7570270755949915 days and will go to Rehab
Stroke patient 4 arrived at 4.873907741841016 days and will go to Other
TIA patient 4 finished treatment at 5.624910741551146 days
Stroke patient 5 arrived at 6.800875823119226 days and will go to Rehab
Stroke patient 5 finished treatment at 6.833127949694689 days
Other patient 6 arrived at 7.638231620087317 days
10
